Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [6]:
from random import random, choice, choices, randint, sample, shuffle
from dataclasses import dataclass
from copy import deepcopy
import numpy as np
import lab9_lib

from tqdm import trange
import math
from itertools import combinations

In [7]:
NUM_GEN=1000
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .15
NUM_ISLANDS=8
NUM_POPULATION=200
NUM_OFFSPRING=300
LOCI = 1000
NUM_MIGRANTS = 25
MIGRATION_STEP = 50


In [8]:
@dataclass
class Individual:
    genotype: list  # List representing an individual's genetic code.
    fitness: float  # Fitness score of the individual.

    def __init__(self, genotype=None):
        # Initialize an Individual object
        
        if genotype is None:
            self.genotype = choices([0, 1], k=1000)
        else:
            self.genotype = genotype

    def clone(self):
        # Create and return a clone of the individual

        return Individual(fitness=self.fitness, genotype=self.genotype)

    def mutate(self):
        # Perform mutation on the individual's genotype

        mutated_genotype = deepcopy(self.genotype)
        index = choice(range(len(self.genotype)))
        if self.genotype[index] == 1:
            mutated_genotype[index] = 0
        else:
            mutated_genotype[index] = 1
        return Individual(genotype=mutated_genotype)


def select_parent(pop):
    # Select a parent from the population using tournament selection
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion


def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    # Perform one-cut crossover between two individuals
    cut_point = randint(0, NUM_GEN-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == NUM_GEN
    return offspring


def n_cut_xover(ind1: Individual, ind2: Individual, n=4) -> Individual:
    # Perform n-cut crossover between two individuals

    cut_points = sample(range(1, NUM_GEN-1), n)
    cut_points.sort()
    genotype = []
    for i in range(n+1):
        if i == 0:
            genotype += ind1.genotype[:cut_points[i]]
        elif i != n:
            if i % 2 == 0:
                genotype += ind1.genotype[cut_points[i-1]:cut_points[i]]
            else:
                genotype += ind2.genotype[cut_points[i-1]:cut_points[i]]
        else:
            if i % 2 == 0:
                genotype += ind1.genotype[cut_points[i-1]:]
            else:
                genotype += ind2.genotype[cut_points[i-1]:]

    offspring = Individual(fitness=None,
                           genotype=genotype)
    return offspring


def tournament_selection(population: list[Individual]) -> Individual:
    # Select an individual from the population using tournament selection

    pool = choices(population, k=TOURNAMENT_SIZE)
    champion = max(pool, key=lambda i: i.fitness)
    return champion


def uniform_xover(ind1: Individual, ind2: Individual) -> Individual:
    # Perform uniform crossover between two individuals

    offspring_genotype = [ind1.genotype[i] if random() < 0.5 else ind2.genotype[i] for i in range(LOCI)]
    return Individual(genotype=offspring_genotype)


In [ ]:
def algorithm_island(problem_type: int, parent_selection: callable, xover: callable) -> Individual:
    ## Island model genetic algorithm for optimization problems
    
    # Set up the fitness function based on the problem type
    fitness = lab9_lib.make_problem(problem_type)

    # Initialize populations for each island with random individuals
    populations = [[Individual() for _ in range(NUM_POPULATION)] for _ in range(NUM_ISLANDS)]

    # Evaluate fitness and sort individuals in each population
    for population in populations:
        for i in population:
            i.fitness = fitness(i.genotype)
        population.sort(key=lambda i: i.fitness, reverse=True)

    # Initialize best individuals and the best global individual
    best_individuals = [population[0] for population in populations]
    best_global_individual = max(best_individuals, key=lambda x: x.fitness)

    # Iterate over generations
    pbar = trange(0, NUM_GEN)
    for generation in pbar:
        # Migration between islands
        if (generation + 1) % MIGRATION_STEP == 0:
            shuffle(populations)
            for idx in range(0, NUM_ISLANDS - 1, 2):
                tmp = populations[idx][:NUM_MIGRANTS]
                populations[idx][:NUM_MIGRANTS] = populations[idx + 1][:NUM_MIGRANTS]
                populations[idx + 1][:NUM_MIGRANTS] = tmp

        # Iterate over each island
        for i in range(NUM_ISLANDS):
            pbar.set_description(f"Best-individual across island fitness: {best_global_individual.fitness}")

            # Break if the best individual in the island has fitness close to 1
            if math.isclose(1, populations[i][0].fitness):
                break

            # Generate offspring
            offspring = list()
            for _ in range(NUM_OFFSPRING):
                if random() < MUTATION_PROBABILITY:
                    # Mutation
                    p = parent_selection(populations[i])
                    o = p.mutate()
                else:
                    # Crossover and mutation
                    p1 = parent_selection(populations[i])
                    p2 = parent_selection(populations[i])
                    o = xover(p1, p2).mutate()

                offspring.append(o)

            # Evaluate fitness of offspring
            for ind in offspring:
                ind.fitness = fitness(ind.genotype)

            # Add offspring to the population, sort, and keep the top individuals
            populations[i].extend(offspring)
            populations[i].sort(key=lambda ind: ind.fitness, reverse=True)
            populations[i] = populations[i][:NUM_POPULATION]

            # Update the best individual in the island
            if populations[i][0].fitness > best_individuals[i].fitness:
                best_individuals[i] = populations[i][0]

            # Update the best global individual
            if best_individuals[i].fitness > best_global_individual.fitness:
                best_global_individual = best_individuals[i]

    # Print and return the result
    print(
        f'Problem {problem_type}\nFitness: {best_global_individual.fitness}\nSolved with {fitness.calls:,} fitness calls'
    )
    return best_global_individual

# Run the algorithm for different optimization problems
best_individual_1 = algorithm_island(1, tournament_selection, uniform_xover)


In [ ]:
best_individual_2 = algorithm_island(2, tournament_selection, uniform_xover)

In [ ]:
best_individual_5 = algorithm_island(5, tournament_selection, uniform_xover)

In [ ]:
best_individual_10 = algorithm_island(10, tournament_selection, uniform_xover)